In [3]:
import numpy as np
import tifffile
from scipy import ndimage
from skimage.filters import gaussian, threshold_otsu
from skimage.measure import label
import os
import glob
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/Static-A-1/Golgi'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Golgi'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Find all .tif files in the input directory
input_files = glob.glob(os.path.join(input_dir, '*.tif'))
print(f"Found {len(input_files)} .tif files to process")

# Process each file
for input_file in input_files:
    # Get base filename for output
    filename = os.path.basename(input_file)
    base_name = os.path.splitext(filename)[0]

    print(f"Processing: {filename}")

    # Load the image
    img = tifffile.imread(input_file)
    print(f"  Image shape: {img.shape}")

    # Extract Golgi channel based on image format
    if len(img.shape) == 2:
        # Single channel image (already Golgi)
        print("  Detected single-channel image, using as is")
        golgi_channel = img
    elif len(img.shape) == 3 and img.shape[0] == 3:
        # Format is (C, H, W)
        print("  Detected format: (C, H, W)")
        golgi_channel = img[2]  # Third channel
    elif len(img.shape) == 3 and img.shape[2] == 3:
        # Format is (H, W, C)
        print("  Detected format: (H, W, C)")
        golgi_channel = img[:, :, 2]  # Third channel
    else:
        print(f"  Unexpected image shape: {img.shape}. Using first channel/plane.")
        if len(img.shape) == 3:
            golgi_channel = img[0] if img.shape[0] < img.shape[1] else img[:, :, 0]
        else:
            golgi_channel = img


    # Apply Gaussian blur for noise reduction
    print("  Applying Gaussian blur...")
    sigma = 1.0  # Adjust sigma value based on your needs
    golgi_smoothed = gaussian(golgi_channel, sigma=sigma)
    print(f"  Gaussian blur applied with sigma={sigma}")

    # Apply Otsu thresholding
    print("  Performing Otsu thresholding...")
    thresh = threshold_otsu(golgi_smoothed)
    binary = golgi_smoothed > thresh

    # Apply erosion to remove small noise
    print("  Applying erosion...")
    eroded = ndimage.binary_erosion(binary, iterations=3)

    # Apply opening (erosion followed by dilation) to smooth boundaries
    print("  Applying opening...")
    opened = ndimage.binary_opening(eroded, iterations=1)

    # Label connected components
    print("  Labeling connected components...")
    labels = label(opened)
    print(f"  Segmentation complete. Found {len(np.unique(labels))-1} objects")

    # Save the different outputs

    # Save the labeled image (main output) - this is the only output we'll keep
    labels_output = os.path.join(output_dir, f"{base_name}_golgi_labels.tif")
    tifffile.imwrite(labels_output, labels.astype(np.uint32))
    print(f"  Saved labeled image to {labels_output}")

    # Print some statistics about the segmentation
    print(f"  Threshold value used: {thresh}")
    print(f"  Found {len(np.unique(labels))-1} Golgi objects")
    print(f"  Processing complete for {filename}\n")

print("All processing complete!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 16 .tif files to process
Processing: denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Golgi_contrast_bg_tophat.tif
  Image shape: (1024, 1024)
  Detected single-channel image, using as is
  Applying Gaussian blur...
  Gaussian blur applied with sigma=1.0
  Performing Otsu thresholding...
  Applying erosion...
  Applying opening...
  Labeling connected components...
  Segmentation complete. Found 320 objects
  Saved labeled image to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Golgi/denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Golgi_contrast_bg_tophat_golgi_labels.tif
  Threshold value used: 0.16513406871803232
  Found 320 Golgi objects
  Processing complete for denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Golgi_contrast_bg_tophat.tif

Processing: denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_Golgi_contrast_bg_tophat.